## Module Import 

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
import os
import time as t
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.layers import Dense
np.set_printoptions(precision=8)
seed=99

print(tf. __version__)

2.7.0


In [ ]:
# change the directory to data location
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN')

Mounted at /content/drive


In [ ]:
# Read the file
file = 'molar_fraction_major_chemical_marker.hdf5'
f = h5py.File(file, 'r')

In [ ]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('molar_fraction'),dtype =np.float32)
print(dataset.shape)

(50901851, 6)


In [ ]:
# Separate the feature and the label
X_old = dataset[:,:-1]
Y = dataset[:,[-1]]
print(X_old.shape,Y.shape)

(50901851, 5) (50901851, 1)


In [ ]:
# the product of mass fraction and density to find mole fraction.
# get the mass fraction
y_nh = X_old[:,0:1]
y_nh2 = X_old[:,1:2]
y_oh = X_old[:,2:3]
# Get the tmperature{The additional input}
temp = X_old[:,4:5]
# get the density
density = X_old[:,3:4]

# A linear relation between mass fractio and mole fraction
X_nh = y_nh *density
X_nh2 = y_nh2 *density
X_oh = y_oh *density

# The new input
X = np.c_[X_nh,X_nh2,X_oh,temp]
print(X)

[[3.24108413e-07 7.15551323e-06 1.50022985e-04 1.88831458e+03]
 [3.24264960e-07 7.15909437e-06 1.50018735e-04 1.88830969e+03]
 [3.24387145e-07 7.16189152e-06 1.50015388e-04 1.88830579e+03]
 ...
 [1.03291832e-05 3.26863956e-05 8.84004578e-04 2.29063330e+03]
 [1.01743235e-05 3.21374937e-05 8.88756535e-04 2.29142432e+03]
 [9.85278712e-06 3.10329524e-05 8.92953656e-04 2.29242944e+03]]


In [ ]:
#shuffle the dataset
X,Y = shuffle(X, Y, random_state=seed)
print(X, Y)

[[1.1436125e-05 6.6332905e-05 9.4569393e-04 1.8394366e+03]
 [2.3975999e-05 3.6371115e-04 1.9834892e-04 1.5187620e+03]
 [5.1061324e-05 5.8826891e-04 1.4799622e-04 1.7107389e+03]
 ...
 [3.0624233e-05 4.4022195e-04 2.0987146e-04 1.5530289e+03]
 [4.0838397e-05 2.8424858e-04 6.6376093e-04 1.8717084e+03]
 [3.5612813e-06 1.4560942e-04 2.2836184e-05 1.4690447e+03]] [[2.6408230e+09]
 [4.5877038e+09]
 [8.5497395e+09]
 ...
 [6.0870994e+09]
 [8.8248003e+09]
 [2.3323902e+08]]


In [ ]:
# Normalization
epsilon = 1e-10
Y_n = (Y - np.min(Y) + epsilon  *(np.max(Y)-np.min(Y)))/ (np.max(Y)-np.min(Y) + epsilon   * (np.max(Y)-np.min(Y)))
X_n = (X - np.min(X, axis=0) + epsilon  *(np.max(X, axis=0)-np.min(X, axis=0)))/ (np.max(X, axis=0)-np.min(X, axis=0) + epsilon  * (np.max(X, axis=0)-np.min(X, axis=0)))

In [ ]:
# # create the normalization fit 
# scale_X = MinMaxScaler()
# scale_Y = MinMaxScaler()

# # The fit function
# print(scale_X.fit(X))
# print(scale_Y.fit(Y))
print(X_n,Y_n)

[[0.12565003 0.0882831  0.5462989  0.58453494]
 [0.26346436 0.48807305 0.1142474  0.37687802]
 [0.5611345  0.78996444 0.08513774 0.5011951 ]
 ...
 [0.33652905 0.5909328  0.12090876 0.39906803]
 [0.44878364 0.38124502 0.3833092  0.6054329 ]
 [0.0391048  0.19486102 0.01278082 0.34468296]] [[0.13353658]
 [0.23251079]
 [0.43393013]
 ...
 [0.30873606]
 [0.44791347]
 [0.01114142]]


In [ ]:
#test_train split 
X_train, X_test, Y_train, Y_test = train_test_split(X_n, Y_n, test_size=0.1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(45811665, 4) (5090186, 4) (45811665, 1) (5090186, 1)


In [ ]:
# # normalize the train data and label
# X_train_n = scale_X.transform(X_train)
# Y_train_n = scale_Y.transform(Y_train)
# print(X_train_n)
# print(Y_train_n)

In [ ]:
# Create the chunk 
X_chunk = tf.Variable(X_train)
Y_chunk = tf.Variable(Y_train)
print(X_chunk, Y_chunk)

<tf.Variable 'Variable:0' shape=(45811665, 4) dtype=float32, numpy=
array([[0.47892272, 0.27262077, 0.25037894, 0.7927561 ],
       [0.6009717 , 0.8278306 , 0.08405775, 0.5186661 ],
       [0.51237583, 0.32715425, 0.17693868, 0.7906615 ],
       ...,
       [0.07583844, 0.37335113, 0.01106486, 0.11364169],
       [0.35645086, 0.18642567, 0.2318291 , 0.8985015 ],
       [0.13512753, 0.18524146, 0.12005832, 0.64959687]], dtype=float32)> <tf.Variable 'Variable:0' shape=(45811665, 1) dtype=float32, numpy=
array([[0.27378303],
       [0.4709165 ],
       [0.20956801],
       ...,
       [0.0695335 ],
       [0.09740939],
       [0.02494465]], dtype=float32)>


In [ ]:
# Creaate the batch slices.
train_data = tf.data.Dataset.from_tensor_slices((X_chunk, Y_chunk))
train_data = train_data.batch(2048, drop_remainder=True)
print(train_data)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# test_data = test_data.batch(1024, drop_remainder=True)

<BatchDataset shapes: ((2048, 4), (2048, 1)), types: (tf.float32, tf.float32)>


In [ ]:
# Get the pretrained model
# Read the file
file = 'model_with_molar_fraction_major_CM.h5'
model = tf.keras.models.load_model(file)

# Get summary
model.summary()

Model: "Heat_release_rate"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h_layer1 (Dense)            (None, 40)                200       
                                                                 
 h_layer2 (Dense)            (None, 24)                984       
                                                                 
 h_layer3 (Dense)            (None, 24)                600       
                                                                 
 output_layers (Dense)       (None, 1)                 25        
                                                                 
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The Ki the is respective exponents of the species massfraction and temperature, which could yeild us a perfect reconstruction of the HRR
k1,k2,k3,k4 = tf.Variable(-1.0),tf.Variable(-1.0),tf.Variable(-0.75),tf.Variable(1.3)

In [ ]:
# The customized mean square error, taking the batch data at any instance
def customized_MSE(true, pred):
    MSE = tf.reduce_mean(tf.square(true-pred))
    return MSE
# The customized loss taking the non-linear equation as residual
def total_mse(MSE_NN, X, Y, k1, k2, k3, k4):# the MSE_NN is the mean squared error of the neural network 
    residual = tf.square(tf.math.log(Y) - (k1*tf.math.log(X[:,0:1]) + k2* tf.math.log(X[:,1:2]) + k3* tf.math.log(X[:,2:3]) +  tf.math.log(k4)))
    MSE_func = tf.reduce_mean(residual)
    total =  MSE_NN + MSE_func
    return total

In [ ]:
# Optimizer 
# optimizers = tf.keras.optimizers.SGD(learning_rate=0.3,momentum=0.8)
optimizers = tf.keras.optimizers.Adam(
    learning_rate=0.03, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

In [ ]:
#Training with customized optimization technic (Automatic-Gradient)
epochs = 25
#lr_rate = 0.03

for epoch in range(epochs):
  print("Starting a new epoch",epoch+1)
  for step, (X_batch, Y_batch) in enumerate(train_data):
    with tf.GradientTape(persistent=True) as tape:
      Y_pred = model(X_batch)
      current_loss = customized_MSE(Y_batch,Y_pred)
      new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3, k4)

    # The gradient check  
    gradient = tape.gradient(new_loss, [k1,k2,k3,k4])
    grad = tape.gradient(current_loss, model.trainable_variables)

    #Assign new variables to the model using optimizer isntead of sub assign
    optimizers.apply_gradients(zip(grad, model.trainable_variables))
    optimizers.apply_gradients(zip(gradient, [k1,k2,k3,k4]))
  
  print("k1:{} k2:{} k3:{} k4:{}".format(k1.numpy(),k2.numpy(),k3.numpy(),k4.numpy()))
  print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.19341745972633362 k2:0.8483633995056152 k3:0.5420240163803101 k4:1.7706117630004883
current_Loss: 5.266994048724882e-05 new_loss: 0.27659377455711365
Starting a new epoch 2
k1:0.1934175193309784 k2:0.8483633995056152 k3:0.5420240163803101 k4:1.7706117630004883
current_Loss: 5.519338446902111e-05 new_loss: 0.2765963077545166
Starting a new epoch 3
k1:0.19341757893562317 k2:0.8483633399009705 k3:0.5420240163803101 k4:1.7706116437911987
current_Loss: 4.2372346797492355e-05 new_loss: 0.2765834927558899
Starting a new epoch 4
k1:0.19341757893562317 k2:0.8483633399009705 k3:0.5420240163803101 k4:1.7706115245819092
current_Loss: 5.122112634126097e-05 new_loss: 0.27659234404563904
Starting a new epoch 5
k1:0.19341754913330078 k2:0.8483633995056152 k3:0.5420239567756653 k4:1.7706116437911987
current_Loss: 5.733103898819536e-05 new_loss: 0.27659842371940613
Starting a new epoch 6
k1:0.19341742992401123 k2:0.8483633995056152 k3:0.5420240163803101 k4:1.7706117630004883
